# Locating the end of the News Reporter's Speech

This Jupyter Notebook implements a video processing pipeline to detect semantic transition points (cut points) in videos using the CLIP (Contrastive Language-Image Pretraining) model. It processes a directory of videos, identifies the first frame where its semantic is largely different from the first frame, and saves the results to a pickle file.

In [ ]:
import cv2
import torch
import clip
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import os
from tqdm import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"


model, preprocess = clip.load("ViT-L/14@336px", device=device)

def get_frame_similarity(frame1, frame2):
    """
    Use the CLIP model to calculate semantic similarity between two frames.
    """
    frame1 = preprocess(Image.fromarray(frame1)).unsqueeze(0).to(device)
    frame2 = preprocess(Image.fromarray(frame2)).unsqueeze(0).to(device)
    with torch.no_grad():
        features1 = model.encode_image(frame1)
        features2 = model.encode_image(frame2)
    return torch.cosine_similarity(features1, features2).item()

def find_cut_point(video_path, similarity_threshold=0.8):
    """
    Apply a binary search method to find the transition points of semantic changes in the video and return the results by frame number.
    """
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    
    ret, base_frame = cap.read()
    if not ret:
        print(f"Failed to read the video: {video_path}")
        return None

    start_frame = 0
    end_frame = total_frames - 1
    cut_frame = None

    while start_frame <= end_frame:
        mid_frame = (start_frame + end_frame) // 2
        cap.set(cv2.CAP_PROP_POS_FRAMES, mid_frame)
        ret, mid_frame_img = cap.read()
        if not ret:
            break

        similarity = get_frame_similarity(base_frame, mid_frame_img)
        

        if similarity < similarity_threshold:
            cut_frame = mid_frame  
            end_frame = mid_frame - 1  
        else:
            start_frame = mid_frame + 1  

    cap.release()

    
    if cut_frame is None:
        cut_frame = 0
        has_problem = True
    else:
        has_problem = False

    return cut_frame, has_problem

def process_videos_in_directory(directory, similarity_threshold=0.8):
    """
    Iterate through all videos in the specified directory, find the segmentation points, and save them to a DataFrame
    """
    results = []
    video_files = [f for f in os.listdir(directory) if f.endswith(".mp4")]

    
    for filename in tqdm(video_files, desc="Processing Videos"):
        video_path = os.path.join(directory, filename)

        
        cut_frame, has_problem = find_cut_point(video_path, similarity_threshold)

        
        results.append({
            "video_name": filename,
            "cut_frame": cut_frame,
            "has_problem": has_problem
        })

    
    df = pd.DataFrame(results)
    return df


if __name__ == "__main__":
    video_directory = "./TVB Reporter Videos"
    similarity_threshold = 0.85

    
    df = process_videos_in_directory(video_directory, similarity_threshold)

    
    output_path = os.path.join(video_directory, "cut_points.pkl")
    df.to_pickle(output_path)

    print(f"Processing completed. Results have been saved to: {output_path}")
    print(df)


Processing Videos: 100%|██████████| 108050/108050 [30:45:23<00:00,  1.02s/it]  

Processing completed. Results have been saved to: ./TVB Reporter Videos/cut_points.pkl
                          video_name  cut_frame  has_problem
0       59654a72e6038331360802e0.mp4        235        False
1       59655ff4e603831f360802e1.mp4        151        False
2       596566e7c5e16c5133bc8476.mp4        153        False
3       59656decc5e16c4d33bc8475.mp4        299        False
4       59656ed0c5e16c5433bc8475.mp4        170        False
...                              ...        ...          ...
108045  678cc14d591bed663d6f86ac.mp4        231        False
108046  678ccd28591bed663d6fe2c2.mp4        669        False
108047  678cd118591bed663d6ffc9e.mp4        999        False
108048  678cd46e591bed663d7014d5.mp4        546        False
108049  678cd7a1591bed663d702db4.mp4        400        False

[108050 rows x 3 columns]
